In [ ]:
import os
import sys
from dotenv import load_dotenv
from typing import Optional
from langchain_community.vectorstores import SupabaseVectorStore
from supabase import create_client
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings

load_dotenv()

# Supabase client
supabase_url = os.getenv("SUPABASE_URL")
supabase_key = os.getenv("SUPABASE_KEY")
supabase = create_client(supabase_url, supabase_key)

# Embeddings
embeddings = AzureOpenAIEmbeddings(
    model=os.getenv("AZURE_EMBEDDING_MODEL_NAME"),
    azure_endpoint=os.getenv("AZURE_EMBEDDING_ENDPOINT"),
    #openai_api_key=os.getenv("AZURE_EMBEDDING_API_KEY"),
    openai_api_version="2024-02-01",
)


### Load the Docs

In [7]:

# Path to FAA regulations folder
FAA_FOLDER = "./_regulations/FAA_USA"

def build_vectorstore_faa() -> Optional[SupabaseVectorStore]:
    """Load FAA PDFs, chunk them, and build a Supabase vectorstore in regulation_embd."""
    folder = FAA_FOLDER
    if not os.path.isdir(folder):
        print(f"[INFO] No folder found at '{folder}'.", file=sys.stderr)
        return None

    print(f"[LOAD] Loading PDFs from: {folder}")
    loader = DirectoryLoader(
        folder,
        glob="**/*.pdf",
        loader_cls=PyPDFLoader,
        show_progress=True
    )
    documents = loader.load()
    if not documents:
        print(f"[INFO] No documents found in {folder}", file=sys.stderr)
        return None

    # Split into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)

    # Add regulator_code metadata
    for doc in docs:
        doc.metadata["regulator_code"] = "faa"

    print(f"[CHUNKS] Prepared {len(docs)} chunks for FAA regulations")

    # Push into Supabase
    vs = SupabaseVectorStore.from_documents(
        docs,
        embeddings,
        client=supabase,
        table_name="regulation_embd",
        query_name="match_documents",  # safer to set explicitly
    )
    return vs

vs = build_vectorstore_faa()
if vs:
    print("[SUCCESS] FAA embeddings uploaded to Supabase table: regulation_embd")
else:
    print("[FAILED] Could not build vectorstore")


[LOAD] Loading PDFs from: ./_regulations/FAA_USA


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:27<00:00,  9.17s/it]


[CHUNKS] Prepared 656 chunks for FAA regulations
[SUCCESS] FAA embeddings uploaded to Supabase table: regulation_embd


### Doc retriver

In [20]:
from langchain_openai import AzureChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import SupabaseVectorStore

# Same supabase client + embeddings from before
vs = SupabaseVectorStore(
    client=supabase,
    embedding=embeddings,
    table_name="regulation_embd",
    query_name="match_documents"  # must match your SQL function if defined
)

retriever = vs.as_retriever(search_kwargs={"k": 5})
'''
llm = AzureChatOpenAI(
    deployment_name=os.getenv("AZURE_CHAT_MODEL_NAME"),
    openai_api_key=os.getenv("AZURE_CHAT_API_KEY"),
    openai_api_version="2024-02-01",
    azure_endpoint=os.getenv("AZURE_CHAT_ENDPOINT"),
)
'''
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
    temperature=0
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# query = "What are the TLOF size requirements according to FAA?"
# query = "What is MTOW?"
# query = "Takeoff and Landing Area Minimum Dimensions for TLOF?"
# query = "Flight Path Alignment?"
# query = "Perimeter Lighting Intensity and Distribution from Approach Angle 0 to 90 degrees?"
# query = "What are the requirements for a square TLOF?"
query = "what are safety oversight by the GCAA following the issue of a heliport / vertiport certificate."

"""
query = You are given a list of shape dimension properties. For each property, provide the following information in a structured table format:

Property Name
Default Value (if specified)
Minimum Value (if specified, otherwise N/A)
Maximum Value (if specified, otherwise N/A)

Here are the properties:
diameter: number
isVisible: boolean
layerName: string
shapeType: string
width: number
length: number
sides: integer
diameter: number
height: number
baseHeight: number
rotation: number
transparency: number
Please output the results as a table with columns:
Property | Default | Min | Max"""

result = qa_chain.invoke(query)

print("Answer:", result["result"])
print()  # adds a blank line
print("Sources:", [doc.metadata for doc in result["source_documents"]])

Answer: The safety oversight by the GCAA following the issuance of a heliport or vertiport certificate includes:

1. Imposing operating restrictions and/or sanctions at a heliport/vertiport if necessary.
2. Conducting further audits/inspections to ensure continued compliance with regulatory and critical safety elements.
3. Monitoring the heliport/vertiport to ensure it operates in accordance with the conditions endorsed on the certificate.
4. Reassessing previously issued approvals or permissions, especially if there are delays or extended periods of inactivity.
5. Ensuring that the operator maintains arrangements with the AIS to keep safety-related conditions of the heliport/vertiport up-to-date and published.

If non-compliance or unresolved safety deficiencies are identified, the GCAA may restrict, suspend, or revoke the certificate as per the provisions outlined in the regulations.

Sources: [{'page': 51, 'title': 'Microsoft Word - CAR VFI for Publication_29_05_2025', 'author': 'Fa